In [3]:
import pandas as pd
from datetime import timedelta

In [7]:
# Load prescription data 
df = pd.read_excel("updated_bmi_data.xlsx")

# Parse prescription date and extract year
df["date_prescribed"] = pd.to_datetime(df["date_prescribed"], errors="coerce")
df["Year"] = df["date_prescribed"].dt.year
df["Month"] = df["date_prescribed"].dt.month

# Clean key numeric fields
df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce")
df["duration"] = pd.to_numeric(df["duration"], errors="coerce")
df["Strength (mg)"] = pd.to_numeric(df["Strength (mg)"], errors="coerce")

# Compute per-prescription dose (quantity × strength)
df["Total_Dose_mg"] = df["quantity"] * df["Strength (mg)"]

# Create prescription windows (still useful for optional duration analysis)
df["start_date"] = df["date_prescribed"]
df["end_date"] = df["start_date"] + pd.to_timedelta(df["duration"], unit="D")

# Calculate non-overlapping durations
def calculate_non_overlapping_days(intervals):
    parsed = [(pd.to_datetime(start), pd.to_datetime(end)) 
              for start, end in intervals 
              if pd.notnull(start) and pd.notnull(end)]
    parsed.sort(key=lambda x: x[0])

    merged = []
    for start, end in parsed:
        if not merged or start > merged[-1][1]:
            merged.append([start, end])
        else:
            merged[-1][1] = max(merged[-1][1], end)

    return sum((end - start).days for start, end in merged)

adjusted_duration = (
    df.groupby(["patID", "Year"])
    .apply(lambda x: calculate_non_overlapping_days(x[["start_date", "end_date"]].values.tolist()))
    .reset_index(name="Adjusted_Duration_Yearly")
)

# Aggregate raw yearly totals 
yearly_summary = (
    df.groupby(["patID", "Year"]).agg({
        "Total_Dose_mg": "sum",
        "quantity": "sum",
        "duration": "sum"
    }).reset_index()
)

yearly_summary.columns = [
    "patID", "Year",
    "Total_Yearly_Dose",
    "Total_Quantity_Yearly",
    "Total_Duration_Yearly"
]

# Merge totals and adjusted duration
df = df.merge(yearly_summary, on=["patID", "Year"], how="left")
df = df.merge(adjusted_duration, on=["patID", "Year"], how="left")

# Load and merge hospital admissions 
admissions = pd.read_excel("hospital_admissions.xlsx")
merged_patient_data = df.merge(admissions, on=["patID", "Year"], how="left")

# Flag rows with admission data
merged_patient_data["has_admitdate"] = merged_patient_data["admitdate"].notna().astype(int)

# Sort to prioritize admission records, then latest script
merged_patient_data = merged_patient_data.sort_values(
    by=["patID", "Year", "has_admitdate", "date_prescribed"],
    ascending=[True, True, False, False]
)

# Keep one record per patient-year
merged_patient_data = merged_patient_data.drop_duplicates(subset=["patID", "Year"], keep="first")
merged_patient_data = merged_patient_data.drop(columns=["has_admitdate"])

# Final cleanup and derived fields
merged_patient_data["admitted"] = merged_patient_data["admitted"].fillna("No")
merged_patient_data["DeathDate"] = merged_patient_data["DeathDate"].fillna("N/A")

merged_patient_data["Year_of_event"] = pd.to_datetime(merged_patient_data["date_prescribed"], errors="coerce").dt.year
merged_patient_data["age_at_event"] = merged_patient_data["Year_of_event"] - merged_patient_data["birthyear"]

# Save cleaned output
merged_patient_data.to_excel("yearly_merged_data_x.xlsx", index=False)
print("Cleaned dataset saved as 'yearly_merged_data_x.xlsx'")


/var/folders/dr/54f_3qc542jcqd08fgp715200000gn/T/ipykernel_5627/3554543553.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: calculate_non_overlapping_days(x[["start_date", "end_date"]].values.tolist()))


Cleaned dataset saved as 'yearly_merged_data_x.xlsx'
